In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
df = pd.read_csv("./Dataset/Parking_Violations_Issued_-_Fiscal_Year_2020.csv", sep=';')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,18,20,22,23,26,27,29,30,31,32,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(df.columns[16], df.columns[17], df.columns[19], df.columns[21], df.columns[22], df.columns[1])

Issuer Code Issuer Command Violation Time Violation County Violation In Front Of Or Opposite Plate ID


In [5]:
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1447152396,JET2661,NY,PAS,06/28/2019,21,SDN,BMW,P,27390,...,GRY,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
1,1447152402,JCV6523,NY,PAS,06/28/2019,20,SDN,TOYOT,P,36290,...,GY,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
2,1447152554,GMK6954,NY,PAS,06/16/2019,19,SUBN,BMW,P,36270,...,BLK,0.0,2019,-,0,NaN,NaN,NaN,NaN,NaN
3,1447152580,JGX1641,NY,PAS,06/24/2019,19,SDN,AUDI,P,36270,...,BLK,0.0,2015,-,0,NaN,NaN,NaN,NaN,NaN
4,1447152724,GDM8069,NY,COM,07/06/2019,48,NaN,NaN,P,31190,...,NaN,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN


# Columns

In [8]:
df.columns.tolist()

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Issuing Agency',
 'Street Code1',
 'Street Code2',
 'Street Code3',
 'Vehicle Expiration Date',
 'Violation Location',
 'Violation Precinct',
 'Issuer Precinct',
 'Issuer Code',
 'Issuer Command',
 'Issuer Squad',
 'Violation Time',
 'Time First Observed',
 'Violation County',
 'Violation In Front Of Or Opposite',
 'House Number',
 'Street Name',
 'Intersecting Street',
 'Date First Observed',
 'Law Section',
 'Sub Division',
 'Violation Legal Code',
 'Days Parking In Effect    ',
 'From Hours In Effect',
 'To Hours In Effect',
 'Vehicle Color',
 'Unregistered Vehicle?',
 'Vehicle Year',
 'Meter Number',
 'Feet From Curb',
 'Violation Post Code',
 'Violation Description',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation']

# Columns with Missing Values

In [9]:
df.columns[df.isna().any()].tolist()

['Plate ID',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Location',
 'Issuer Command',
 'Issuer Squad',
 'Violation Time',
 'Time First Observed',
 'Violation County',
 'Violation In Front Of Or Opposite',
 'House Number',
 'Street Name',
 'Intersecting Street',
 'Sub Division',
 'Violation Legal Code',
 'Days Parking In Effect    ',
 'From Hours In Effect',
 'To Hours In Effect',
 'Vehicle Color',
 'Unregistered Vehicle?',
 'Meter Number',
 'Violation Post Code',
 'Violation Description',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation']

# Columns with Sepcial Format

## Colors
Colors are very badly formatted. They have a specific format specified [here](https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE) but a lot of colors data are not in this format.
**Solution at the end**.

In [10]:
df['Vehicle Color'].dropna().unique().tolist()

['GRY',
 'GY',
 'BLK',
 'BK',
 'ORANG',
 'WH',
 'GR',
 'RD',
 'GRAY',
 'WHT',
 'WT',
 'BLUE',
 'RED',
 'SIL',
 'WHITE',
 'BLU',
 'GREY',
 'SILVE',
 'BLA',
 'GN',
 'BLACK',
 'G',
 'BR',
 'TAN',
 'NH',
 'BRN',
 'BL',
 'GREEN',
 'SL',
 'BLAC',
 'OR',
 'BROWN',
 'WHI',
 'TN',
 'BEIGE',
 'BRW',
 'GRA',
 'SILV',
 'GRN',
 'W',
 'GOLD',
 'SLV/G',
 'YELLO',
 'GLD',
 'B',
 'YW',
 'WHE',
 'NAVY',
 'BREGE',
 'MAR',
 'BW',
 'BUE',
 'GD',
 'BRO',
 'BURGE',
 'BURG',
 'RGOLD',
 'BURGU',
 'GRE',
 'BUL',
 'SLVR',
 'WHTE',
 'WHIT',
 'BIU',
 'S',
 'GOD',
 'BIEGE',
 'SR',
 'PURPL',
 'BLAK',
 'YEL',
 'SLV',
 'GL',
 'BC',
 'YELL',
 'R',
 'PURP',
 'SI',
 'COPPE',
 'SILER',
 'BN',
 'WTE',
 'T',
 'B K',
 'SLI',
 'BROW',
 'BLIK',
 'BUR',
 'OTHER',
 'SLR',
 'BLCK',
 'BG',
 '.',
 'BRONZ',
 'GRE Y',
 'GT',
 'BOLD',
 'MN',
 'SMART',
 'MAROO',
 'BUG',
 'QGRAY',
 'Y',
 'BL/WH',
 'WI',
 'SV',
 'TEAL',
 'ORG',
 'BWN',
 'BE',
 'WT/RE',
 'SIV',
 'BU',
 'UT',
 'SLIVE',
 'BURGN',
 'GK',
 'BLXK',
 'GREN',
 'PUR',
 'YL',
 '//

In [11]:
df[df['Plate ID'] == 'JGX9658']

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
235,1447416624,JGX9658,NY,PAS,06/24/2019,40,SUBN,FORD,P,32633,...,BLUE,0.0,2007,-,0,NaN,NaN,NaN,NaN,NaN
428,1449399691,JGX9658,NY,999,06/05/2019,14,VAN,FORD,P,9540,...,BL,0.0,2001,-,0,NaN,NaN,NaN,NaN,NaN
430,1449399710,JGX9658,NY,PAS,06/04/2019,14,VAN,FORD,P,9540,...,BL,0.0,2007,-,0,NaN,NaN,NaN,NaN,NaN
220210,8701097416,JGX9658,NY,PAS,07/02/2019,83,SUBN,FORD,T,17430,...,BL,NaN,2007,NaN,0,18,NaN,NaN,NaN,NaN
339041,8744115416,JGX9658,NY,PAS,07/02/2019,16,SUBN,FORD,T,17430,...,BL,NaN,2007,NaN,0,01,NaN,NaN,NaN,NaN
1374454,8718607860,JGX9658,NY,PAS,08/01/2019,38,SUBN,FORD,T,34190,...,BL,NaN,2007,NaN,0,07,NaN,NaN,NaN,NaN
1449584,8757462392,JGX9658,NY,PAS,07/29/2019,31,SUBN,FORD,T,34230,...,BL,NaN,2007,NaN,0,07,NaN,NaN,NaN,NaN
1449585,8757462409,JGX9658,NY,PAS,07/29/2019,83,SUBN,FORD,T,34230,...,BL,NaN,2007,NaN,0,07,NaN,NaN,NaN,NaN
1479442,8773651710,JGX9658,NY,PAS,08/08/2019,31,SUBN,FORD,T,32633,...,BL,NaN,2007,NaN,0,99,NaN,NaN,NaN,NaN
2272297,8762358364,JGX9658,NY,PAS,08/09/2019,16,SUBN,FORD,T,17430,...,BL,NaN,2007,NaN,0,99,NaN,NaN,NaN,NaN


*CORRECT COLORS*

In [12]:
df[df['Vehicle Color'].isin(['RD', 'TN', 'WH', 'YW', 'NOCL', 'BL', 'BK', 'BR', 'GL', 'GY', 'MR', 'OR', 'PK', 'PR'])].shape[0] / df.shape[0] * 100

66.5917206824833

In [13]:
df[['color1', 'color2']] = df['Vehicle Color'].str.split(r'[^A-z]', n=1, expand=True)

In [14]:
df[['Vehicle Color', 'color1', 'color2']]

,Vehicle Color,color1,color2
0,GRY,GRY,None
1,GY,GY,None
2,BLK,BLK,None
3,BLK,BLK,None
4,NaN,NaN,NaN
...,...,...,...
4984029,GREY,GREY,None
4984030,BLACK,BLACK,None
4984031,BK,BK,None
4984032,WHITE,WHITE,None


In [15]:
df2 = df.copy()

In [16]:
df2 = df2[df2['color1'].str.upper().isin(['RD', 'TN', 'WH', 'YW', 'NOCL', 'BL', 'BK', 'BR', 'GL', 'GY', 'MR', 'OR', 'PK', 'PR'])]

In [17]:
df2 = df2[df2['color2'].str.upper().isin([None, np.nan, 'RD', 'TN', 'WH', 'YW', 'NOCL', 'BL', 'BK', 'BR', 'GL', 'GY', 'MR', 'OR', 'PK', 'PR'])]

In [22]:
len(df2[df2['color2'].notna()][['Vehicle Color', 'color1', 'color2']])

48

### Solution
As demonstrated above, we should split 'Veh. Color' based on any delimeter that is not a character. There should be two colors at max as specified [here](https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE). Then, any first color that is not undefied shuld be classified as **Unknown**. Unidentified decond colors can either have None or **Unknown**. Of cource we can do better in term of figuring out close matches of colors (e.g., 'BK' and 'BLK' => 'Black') one idea is, for each unknown color we search for the records of the same vehicle to find a record with a known color.

**NOTE**: I need to handle the case when the color has LT and DK modifiers.

***UPDATE***:
As shown in the cell directly above, very few data have second colors. So, to make things simple, only the first color is considered.


## Vehicle Body Type
Use the lookup file. A lot of bad data. If matched, map, otherwise, unknown. If we get new records for an existing vehicle, we update the body type unless unknown.

In [19]:
df2['Vehicle Body Type'].unique().tolist()

['SDN',
 'SUBN',
 nan,
 'P-U',
 'VAN',
 'FOUR',
 'O',
 'OVOR',
 'RV',
 'TRAC',
 'CONV',
 'DELV',
 '4DSD',
 'TR',
 'TLR',
 'RED',
 'OM',
 'TRAI',
 '2DSD',
 'AL',
 'TT',
 'TRL',
 'TWOD',
 'BK',
 'TRUC',
 'PICK',
 'WAGO',
 'TK',
 'CON',
 'AM',
 'TRK',
 'CXM',
 'MC',
 'BUS',
 'ONOI',
 'TAXI',
 'MOT',
 'GY',
 'MINI',
 'NER',
 'OTNE',
 'I',
 '4 DR',
 '2 DR',
 'SW',
 'PKUP',
 'SEDN',
 'LIM',
 'HRSE',
 'H/WH',
 'CP',
 'MCY',
 'WAG',
 'HATC',
 'SUW',
 'UTIL',
 '4DHB',
 'TOW',
 'STAK',
 'DUMP',
 'FLAT',
 'TRLR',
 'UT',
 'CI',
 'L',
 'REFG',
 'SUV',
 'ADP',
 'COMT',
 'CNN',
 'ON',
 'BOX',
 'ANA',
 'TRC',
 'VESP',
 'COUP',
 'OMT',
 'TANK',
 'CXN',
 'SEM',
 'MOTO',
 'REFR',
 'UTL',
 'CM',
 'REFI',
 'COVE',
 'TRD',
 'TRT',
 'TF',
 'PETE',
 'PAT',
 'TCH',
 'APP',
 'RF',
 'TRLI',
 'TRIL',
 'MT',
 'OSO',
 'BIKE',
 'HYUN',
 'TLC',
 'MTR',
 'TRAL',
 'HCY',
 'SEMI',
 'SON',
 'AN',
 'FRT',
 'BW',
 'OTM',
 'SCOO',
 'UTIC',
 'TRUK',
 'BX',
 'RM',
 'REF',
 'CNM',
 'SWT',
 'CIN',
 'OML',
 'FIAT',
 'OI',
 'AR',

## Plate ID's

1. **Records with missing IDs should be drouped**.
2. Plate id can have the same value for differenct vehicle **from different Registration State**. So, conactaning Plate ID and Registration State is a good unique ID!

In [53]:
df3 = df.dropna(subset=['Plate ID'])

In [54]:
df3.drop_duplicates(subset=['Registration State'])

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,color1,color2
0,1447152396,JET2661,NY,PAS,06/28/2019,21,SDN,BMW,P,27390,...,0,-,0,NaN,NaN,NaN,NaN,NaN,GRY,None
9,1447153352,GDH2184,ME,PAS,07/06/2019,48,SDN,DODGE,P,31190,...,0,-,0,NaN,NaN,NaN,NaN,NaN,ORANG,None
11,1447153789,JFW5006,99,PAS,06/16/2019,46,SDN,HONDA,P,36270,...,0,-,0,NaN,NaN,NaN,NaN,NaN,BK,None
14,1447153820,KHW5523,PA,PAS,06/23/2019,78,SDN,DODGE,P,24490,...,0,-,0,NaN,NaN,NaN,NaN,NaN,BK,None
17,1447154411,JKSN62,FL,PAS,06/14/2019,46,SUBN,KIA,P,36270,...,0,-,0,NaN,NaN,NaN,NaN,NaN,BLK,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404829,1454375887,228ZAV,KY,PAS,06/15/2019,46,SUBN,DODGE,P,36880,...,0,-,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427877,1449705431,167049,PE,PAS,07/06/2019,40,SUBN,L/R,P,43730,...,0,-,0,NaN,NaN,NaN,NaN,NaN,WH,None
456891,1459050447,8HRF26,MX,PAS,06/12/2019,9,VAN,ME/BE,P,0,...,0,-,0,NaN,NaN,NaN,NaN,NaN,WH,None
850856,8730131595,280KCM,SK,PAS,07/27/2019,31,PICK,DODGE,T,34690,...,0,NaN,0,45,NaN,NaN,NaN,NaN,GREY,None


## Registration State
Use a lookup of states

## Issuing Agency
Use the information in the dataset page

## Delete Violation Location
Violation Precinct is enough. 
0 in Violation Precinct means Unknown

In [55]:
df3[df3['Violation Location'] != df3['Violation Precinct']]
# df3[(df['Violation Precinct'] > 0) & (df3['Violation Location'] != df3['Violation Precinct'])]

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,color1,color2
117,1449251559,HMA6875,NY,PAS,07/05/2019,46,SDN,DODGE,P,0,...,2007,-,0,NaN,NaN,NaN,NaN,NaN,BK,None
137,1449254901,HXH6615,NY,PAS,07/15/2019,51,SDN,HONDA,P,25830,...,2004,-,0,NaN,NaN,NaN,NaN,NaN,WH,None
207,1449335299,JEF4708,NY,PAS,07/06/2019,48,VAN,HONDA,P,36420,...,0,-,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,1449353162,HSM3720,NY,PAS,06/17/2019,74,SDN,HONDA,P,0,...,2012,-,0,NaN,NaN,NaN,NaN,NaN,GRAY,None
369,1449353174,HSM3720,NY,PAS,06/17/2019,40,SDN,HONDA,P,0,...,2012,-,0,NaN,NaN,NaN,NaN,NaN,GRAY,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826273,8774301445,87142ME,NY,COM,10/07/2019,69,REFG,FRUEH,T,0,...,2014,NaN,0,25,NaN,NaN,NaN,NaN,WH,None
4915852,8785705524,80279ML,NY,COM,10/08/2019,0,NaN,NaN,T,0,...,0,NaN,0,01,NaN,NaN,NaN,NaN,NaN,NaN
4963945,8792400486,67748MH,NY,COM,10/25/2019,69,VAN,FORD,T,0,...,2015,NaN,0,99,NaN,NaN,NaN,NaN,WH,None
4981179,8802650093,13918MJ,NY,COM,10/09/2019,0,NaN,NaN,T,0,...,0,NaN,0,17,NaN,NaN,NaN,NaN,NaN,NaN


## Violation Time
The time is in this format hhmm[A|P]

In [56]:
df3['Violation Time'].unique().tolist()

['1000A',
 '1011A',
 '0107A',
 '0300A',
 '0653A',
 '0515P',
 '0524P',
 '0601P',
 '0935A',
 '1217P',
 '0049A',
 '0046A',
 '0124A',
 '0500A',
 '1221A',
 '1241A',
 '1110A',
 '0105A',
 '1000P',
 '0843A',
 '0955P',
 '0508P',
 '1100A',
 '0718A',
 '0722A',
 '0726A',
 '0430A',
 '0615P',
 '0115A',
 '0350A',
 '0125A',
 '0924A',
 '0918A',
 '0912P',
 '0318P',
 '0850A',
 '1055A',
 '0152A',
 '0845A',
 '0849A',
 '0627A',
 '1101A',
 '0608P',
 '0430P',
 '0917A',
 '0910A',
 '0811A',
 '0748A',
 '1120A',
 '1040A',
 '1143A',
 '1138A',
 '0238P',
 '1201P',
 '1154A',
 '0936A',
 '0405P',
 '1004P',
 '0204P',
 '0523P',
 '0632P',
 '0351P',
 '0550A',
 '0400P',
 '1140A',
 '0148P',
 '0957P',
 '1004A',
 '0123P',
 '0104P',
 '1132A',
 '0550P',
 '1242P',
 '1120P',
 '1058A',
 '0112A',
 '1036A',
 '0503P',
 '0651A',
 '0227P',
 '0805A',
 '0917P',
 '0445A',
 '0855A',
 '1146A',
 '0702P',
 '0410A',
 '0344A',
 '1205A',
 '0926P',
 '1157A',
 '0755A',
 '0732P',
 '0834P',
 '0809P',
 '0734P',
 '0948P',
 '0900P',
 '0700P',
 '0135A',


## Time First Observed
Maybe missing value should be the same as *Violation Time*.

## Date First Observed
Dates are in a bad format!
They are either a '0' which means missing (then maybe make it the same as *Issue Date*) or a bad formatted string. See the below cell for an example.

In [57]:
df3['Date First Observed'][61]

'20,190,625'

## Violation County
A lot of bad data for the list of [Counties](https://simple.wikipedia.org/wiki/List_of_counties_in_New_York). Maybe map them?
- NY, MN => Manhattan
- K, BK => Brooklyn
- BX => Bronx
- R, ST => StatenIsland
- Q, QN => Queens
- ELSE => Unknown

In [58]:
df3['Violation County'].unique().tolist()

['NY', nan, 'R', 'K', 'BX', 'Q', 'QN', 'MN', 'BK', 'ST', 'K   F']

## Violation In Front Of Or Opposite
- I, F => InFront
- O => Opposite
- ELSE => Unknown

In [59]:
df3['Violation In Front Of Or Opposite'].unique().tolist()

['F', 'O', nan, 'I', 'X', 'R', '9']

## House Number
Should be stored with all relevant location information in a dimension table.
    A lot of house numbers with different street names which is wrong obuviously. One solution to this problem is, for each house number find the most common street name, and change all similar house number records accordingly. The problem is that I don't know how to find the correct house numbers. So, ignore it.

## Vehicle Year
A lot of values that are very low (e.g. 0) or very high (> 2021). We should Nullify all values that are less than 1884 (the oldest automobile according to [this page](https://en.wikipedia.org/wiki/La_Marquise)) or more than current year + 1 (a lot of car makers sell new model before the new year).

In [60]:
df3['Vehicle Year'].unique().tolist()

[0,
 2019,
 2015,
 2005,
 2003,
 2004,
 2016,
 2007,
 2017,
 2014,
 2018,
 2001,
 2009,
 2013,
 2006,
 2002,
 1998,
 2008,
 2011,
 1970,
 2012,
 2010,
 1997,
 1973,
 1994,
 1996,
 1999,
 1985,
 1993,
 2021,
 1995,
 1992,
 1980,
 1986,
 1981,
 2065,
 2020,
 2061,
 1989,
 1990,
 1979,
 2066,
 1977,
 1971,
 1987,
 1984,
 1991,
 2049,
 1974,
 2022,
 1988,
 1982,
 2062,
 1983,
 2047,
 2025,
 2063,
 2068,
 2064,
 1978,
 2060,
 2067,
 1972,
 1976,
 1975,
 2069,
 2059,
 2051,
 2057,
 2055,
 2041,
 2045,
 2040,
 2046,
 2042,
 2031,
 2030,
 2044,
 2048,
 2036,
 2053,
 2052,
 2058,
 2054,
 2043,
 2026,
 2023,
 2056,
 2035,
 2034,
 2027,
 2029,
 2032]

## Issuing Agency

This map is taken from the [main dataset page](https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2020/pvqr-7yc4)

A-PORT AUTHORITY, B-TRIBOROUGH BRIDGE AND TUNNEL POLICE, C-CON RAIL, D-DEPARTMENT OF BUSINESS SERVICES, E-BOARD OF ESTIMATE, F-FIRE DEPARTMENT, G-TAXI AND LIMOUSINE COMMISSION, H-HOUSING AUTHORITY, I-STATEN ISLAND RAPID TRANSIT POLICE, J-AMTRAK RAILROAD POLICE, K-PARKS DEPARTMENT, L-LONG ISLAND RAILROAD, M-TRANSIT AUTHORITY, N-NYS PARKS POLICE, O-NYS COURT OFFICERS, P-POLICE DEPARTMENT, Q-DEPARTMENT OF CORRECTION, R-NYC TRANSIT AUTHORITY MANAGERS, S-DEPARTMENT OF SANITATION, T-TRAFFIC, U-PARKING CONTROL UNIT, V-DEPARTMENT OF TRANSPORTATION, W-HEALTH DEPARTMENT POLICE, X-OTHER/UNKNOWN AGENCIES, Y-HEALTH AND HOSPITAL CORP. POLICE, Z-METRO NORTH RAILROAD POLICE, 1-NYS OFFICE OF MENTAL HEALTH POLICE, 2-O M R D D, 3-ROOSEVELT ISLAND SECURITY, 4-SEA GATE ASSOCIATION POLICE, 5-SNUG HARBOR CULTURAL CENTER RANGERS, 6-WATERFRONT COMMISSION OF NY HARBOR, 7-SUNY MARITIME COLLEGE, 9-NYC OFFICE OF THE SHERIFF


**A lot of bad data. Some records have the same Issuer code with differnt Issuing Agent which does not make sense. To solve this, we update the Issuing Agent for each new record unless the new record is *Unknown***.

## Issuer Code
Does not contain any missing values.

In [61]:
len(df3[df3['Issuer Code'].isna()])

0

# Violation Codes Dataset

In [62]:
vc_df = pd.read_excel('./Dataset/ParkingViolationCodes_Nov_2018.xlsx')

In [63]:
vc_df.head()

,VIOLATION CODE,VIOLATION DESCRIPTION,Manhattan 96th St. & below\n(Fine Amount $),All Other Areas\n(Fine Amount $)
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,115,115
